In [1]:
import numpy as np
#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
from keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
import sympy
import numpy as np
import seaborn as sns
import collections

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
ReLU = lambda z: np.maximum(0, z)
sigmoid = lambda z: 1/(1+np.exp(-z))
tanh = lambda z: (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))
softmax= lambda z: np.exp(z) / np.sum(np.exp(z))
class MLP:
    
    def __init__(self, M,layer,act_fun,d_actfun):
        self.M = M
        self.layer=layer
        self.act_fun=act_fun
        self.d_actfun=d_actfun
            
    def fit(self, x, y, optimizer): #input为 60000x784 ，single layer，ReLU activation
        # x=[Nx784]; y=[Nx1]
        N,D = x.shape
        w = np.random.randn(self.M,10) * .1  #w=[128x10] 
        F = np.random.randn(128,128) * .1 #F=[128x128]
        v = np.random.randn(784,self.M) * .1 #v=[784x128]
        o = np.random.randn(784,10) * .1 #针对layer=0，【748x10】

        if self.layer==0:
          params0 = np.array(o)
        elif self.layer==1:
          params0 = [w,v]
        elif self.layer==2:
          params0 = [w,F,v]

        def gradient(x, y, params):
          
          if self.layer==0:
              o = params
              yh = np.dot(x,o)
              yh_n=np.zeros(shape=(N,10))
              for i in range(N):
                yh_n[i]=softmax(yh[i])
              dv=np.dot(x.T,(yh_n-to_categorical(y)))/N #one hot encoding （y）
              dparams = [dv]

          elif self.layer==1:
              w,v = params
              #print(np.dot(x,v).shape)
              z = self.act_fun(np.dot(x,v)) # z=[Nx128]
              yh=np.dot(z,w) # yh=[Nx10]

              for i in range(N):
                yh[i]=softmax(yh[i])

              dy=yh-to_categorical(y) #求dy=yh-y=[Nx10]

              dw = np.dot(z.T,dy)/N #dw=[128x10],跟w size一样，dw=(x.v)^T . (yh-y)
              dz = self.d_actfun(np.dot(x,v))*np.dot(dy, w.T) #dz=[Nx128]           
              dv=np.dot(x.T,dz)/N #dv=[784x128],跟v size一样,dv= x^T . (yh-y).w^T * RELU'
              dparams = [dw,dv]

          elif self.layer==2:
              w,F,v=params
              z = self.act_fun(np.dot(x,v)) # z=[Nx128]
              k = self.act_fun(np.dot(z,F)) # k=[Nx128]
              yh=np.dot(k,w) # yh=[Nx10]
                #yh=np.zeros(shape=(N,10))

              for i in range(N):
                yh[i]=softmax(yh[i])

              y=to_categorical(y)
              dy=yh-y #求dy=yh-y=[Nx10]
              dw = np.dot(k.T,dy)/N #dw=[128x10],跟w size一样，dw=(x.v)^T . (yh-y)
              #dk = self.d_actfun(np.dot(dy, w.T))
              dk = self.d_actfun(np.dot(z, F))*(np.dot(dy, w.T))
              # z = f(xv)
              # k = f(zF)
              # dk/dzF = f'(zF)
              # dzF/dF = z
              # dk/dF = f'(zF)*z
              # dL/dF = dL/dyh * dyh/dkw * dkw/dk * dk/dzF * dzF/dF
              # dL/dF = (yh - y)     * w   * f'(zF) * z

              # dL/dF = dL/dyh * dyh/dkw * dkw/dk * dk/dzF * dzF/dz * dz/dxv * dxv/dv
              # dL/dF = (yh - y)     * w   * f'(zF) * F    * f'(xv) * x

              dF = np.dot(z.T,dk)/N

              dz = np.dot(dk, F.T) * self.d_actfun(np.dot(x,v))
              dv = np.dot(x.T,dz)/N

              dparams = [dw, dF, dv]
              

          return dparams

        
        self.params = optimizer.run(gradient, x, y, params0)
        return self
    
    def predict(self, x): # model prediction,用gradient decent确定好 w 和 v之后，再predict
      N,D = x.shape
      result=[]

      if len(self.params)==2: #layer = 1的predict
        w,v = self.params
        z = self.act_fun(np.dot(x, v)) 
        yh=np.dot(z,w)
        
        for i in range(N):
          yh[i]=softmax(yh[i])
          result.append(np.argmax(yh[i]))
          

      elif len(self.params) == 3: #layer = 2 的 predict
        w,F,v = self.params
        z = self.act_fun(np.dot(x,v)) 
        k = self.act_fun(np.dot(z,F))
        yh = np.dot(k,w)
        
        for i in range(N):
          yh[i]=softmax(yh[i])
          result.append(np.argmax(yh[i]))
      
      else: # layer=0 的 predict
        v = np.array(self.params)
        yh = np.dot(x,v)

        for i in range(N):
          yh[i] = softmax(yh[i])
          result.append(np.argmax(yh[i]))

      result=np.array(result)
      return result

In [4]:
ReLU = lambda z: np.maximum(0, z)
sigmoid = lambda z: 1/(1+np.exp(-z))
tanh = lambda z: (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))
softmax= lambda z: np.exp(z) / np.sum(np.exp(z))

def d_ReLU (x):
  x[x<=0]=0
  x[x>0]=1
  return x  

d_tanh= lambda z: 1-np.square(np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))
d_sigmoid = lambda z: (1/(1+np.exp(-z)))*(1-1/(1+np.exp(-z)))

In [5]:
def evaluate_acc (true,target): # evaluate function，求model准确率
  accuracy = np.sum(true == target) / target.shape[0]
  return accuracy

In [6]:
class GradientDescent: #ta的码没仔细看，call function的时候调整参数：learning_rate和 max_iters
    
    def __init__(self, learning_rate=.001, max_iters=100, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.epsilon = epsilon
        
    def run(self, gradient_fn, x, y, params):
        norms = np.array([np.inf])
        t = 1
        while np.any(norms > self.epsilon) and t < self.max_iters:
#             print(t) #可以comment掉
            
            grad = np.squeeze(gradient_fn(x, y, params))
            #print(grad)
            for p in range(len(params)):
                #print(params[p].shape)
                #print(grad[p].shape)
                params[p] -= self.learning_rate * grad[p]
                
            t += 1
            
            norms = np.array([np.linalg.norm(g) for g in grad])
        return params

In [7]:
(train_X0, train_y), (test_X0, test_y) =  tf.keras.datasets.mnist.load_data() #load data
# flatten 28*28 images to a 784 vector for each image
num_pixels = train_X0.shape[1] * train_X0.shape[2]
train_X0 = train_X0.reshape(train_X0.shape[0], num_pixels).astype('float32')
test_X0 = test_X0.reshape(test_X0.shape[0], num_pixels).astype('float32')
# unit normalization
train_X=train_X0/255.0
test_X=test_X0/255.0




11493376/11490434 [==============================] - 10s 1us/step


In [8]:
# # number of hidden layes: 0,1,2
# layer_n=np.asarray([0,1,2])
# # number of units in each hidden layer: 32,64,128,256
# unit_size=np.asarray([32,64,128,256])

# # optimizer: GradientDescent, 
#             # learning rate: 0.01,0.05,0.09,0.10,0.20,0.50,1.00 
# lrs=np.asarray([0.01,0.05,0.09,0.10,0.20,0.50,1.00])
#             # maximum iterations: 10,50,100,200,500,1000,2000
# iters=np.asarray([10,50,100,200,500,1000,2000])

# for i in range(1,len(layer_n)):
#     n=layer_n[i]
#     print(f'The number of hidden layers is: {n}')
#     for j in range(len(unit_size)):
#         m=unit_size[j]
#         print(f'The number units in each hidden layer is: {m}')
#         model1= MLP(M=m,layer=n,act_fun=ReLU,d_actfun=d_ReLU) #create model1
#         model2= MLP(M=m,layer=n,act_fun=tanh,d_actfun=d_tanh) #create model2
#         model3= MLP(M=m,layer=n,act_fun=sigmoid,d_actfun=d_sigmoid) #create model2
#         for k in range(len(lrs)):
#             rate=lrs[k]
#             print(f'The learning rate of Gradient Descent is: {rate}')
#             for l in range(len(iters)):
#                 iteration=iters[l]
#                 print(f'The maximum number of iterations of Gradient Descent is: {iteration}')
#                 optimizer = GradientDescent(learning_rate=rate, max_iters=iteration) #call optimizer
                
#                 y1= model1.fit(train_X[0:60000], train_y[0:60000], optimizer).predict(test_X)
#                 y2=model2.fit(train_X[0:60000], train_y[0:60000], optimizer).predict(test_X)
#                 y3=model1.fit(train_X[0:60000], train_y[0:60000], optimizer).predict(test_X)
                
#                 accuracy1=evaluate_acc(test_y,y1)
#                 accuracy2=evaluate_acc(test_y,y2)
#                 accuracy3=evaluate_acc(test_y,y3)
                
#                 print(f'The MLP model using ReLU has accuracy: {accuracy1}')
#                 print(f'The MLP model using tanh has accuracy: {accuracy2}')
#                 print(f'The MLP model using sigmoid has accuracy: {accuracy3}')

In [ ]:
# 1 No hidden layer ;
model= MLP(M=0,layer=0,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:60000], train_y[0:60000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(0 hidden layers,0.01 learning rate, 100 maximum iterations) has accuracy: {accuracy}')

# 2 One hidden layer with ReLU; 
model= MLP(M=128,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:60000], train_y[0:60000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(ReLU activation function, 1 hidden layers,128 hidden units, 0.09 learning rate, 100 maximum iterations) has accuracy: {accuracy}')

# 3 Two hidden layers each with ReLU; 
model= MLP(M=128,layer=2,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:60000], train_y[0:60000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(ReLU activation function, 2 hidden layers,128 hidden units, 0.01 learning rate, 100 maximum iterations) has accuracy: {accuracy}')

# 4 Two hidden layers each with tanh; 
model= MLP(M=128,layer=2,act_fun=tanh,d_actfun=d_tanh)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:60000], train_y[0:60000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(tanh activation function, 2 hidden layers,128 hidden units, 0.01 learning rate, 100 maximum iterations) has accuracy: {accuracy}')

# 5 Two hidden layers each with sigmoid; 
model= MLP(M=128,layer=2,act_fun=sigmoid,d_actfun=d_sigmoid)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:60000], train_y[0:60000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(sigmoid activation function, 2 hidden layers,128 hidden units, 0.01 learning rate, 100 maximum iterations) has accuracy: {accuracy}')

# 6 Two hidden layers each with ReLU and add additional L2 regularization; 

# 7 Two hidden layers each with ReLU, with an unnormalized image.
model= MLP(M=128,layer=2,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X0[0:60000], train_y[0:60000], optimizer).predict(test_X0)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(ReLU activation function, 2 hidden layers,128 hidden units, 0.01 learning rate, 100 maximum iterations) has accuracy: {accuracy}')



The MLP model(0 hidden layers,0.01 learning rate, 100 maximum iterations) has accuracy: 0.435


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or s

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or s

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or s

In [ ]:
# compare number of layers
model= MLP(M=64,layer=0,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(0 hidden layers) has accuracy: {accuracy}')

model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(1 hidden layers) has accuracy: {accuracy}')

model= MLP(M=64,layer=2,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(2 hidden layers) has accuracy: {accuracy}')

In [ ]:
# compare number units of layers
model= MLP(M=2,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(2 hidden units) has accuracy: {accuracy}')

model= MLP(M=4,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(4 hidden units) has accuracy: {accuracy}')

model= MLP(M=8,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(8 hidden units) has accuracy: {accuracy}')

model= MLP(M=16,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(16 hidden units) has accuracy: {accuracy}')

model= MLP(M=32,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(32 hidden units) has accuracy: {accuracy}')

model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(64 hidden units) has accuracy: {accuracy}')

model= MLP(M=128,layer=2,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(128 hidden units) has accuracy: {accuracy}')

model= MLP(M=256,layer=2,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(256 hidden units) has accuracy: {accuracy}')

In [ ]:
# compare learning rate
model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.00, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(learning rate=0.00) has accuracy: {accuracy}')

model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(learning rate=0.01) has accuracy: {accuracy}')

model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.02, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(learning rate=0.02) has accuracy: {accuracy}')

model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.05, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(learning rate=0.05) has accuracy: {accuracy}')

model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.09, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(learning rate=0.09) has accuracy: {accuracy}')

model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.10, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(learning rate=0.10) has accuracy: {accuracy}')

In [ ]:
# compare maximum iterations
model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=50)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(maximum iterations=50) has accuracy: {accuracy}')

model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=100)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(maximum iterations=100) has accuracy: {accuracy}')

model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=200)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(maximum iterations=200) has accuracy: {accuracy}')

model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=500)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(maximum iterations=500) has accuracy: {accuracy}')

mmodel= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=1000)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(maximum iterations=1000) has accuracy: {accuracy}')

model= MLP(M=64,layer=1,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.01, max_iters=2000)
y= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)
accuracy=evaluate_acc(test_y,y)
print(f'The MLP model(maximum iterations=2000) has accuracy: {accuracy}')

In [ ]:
# compare train size
model= MLP(M=128,layer=2,act_fun=ReLU,d_actfun=d_ReLU)
optimizer = GradientDescent(learning_rate=0.09, max_iters=1000)
# y1= model.fit(train_X[0:0], train_y[0:0], optimizer).predict(test_X)
y2= model.fit(train_X[0:10], train_y[0:10], optimizer).predict(test_X)
y3= model.fit(train_X[0:100], train_y[0:100], optimizer).predict(test_X)
y4= model.fit(train_X[0:1000], train_y[0:1000], optimizer).predict(test_X)
y5= model.fit(train_X[0:10000], train_y[0:10000], optimizer).predict(test_X)

# accuracy1=evaluate_acc(test_y,y1)
accuracy2=evaluate_acc(test_y,y2)
accuracy3=evaluate_acc(test_y,y3)
accuracy4=evaluate_acc(test_y,y4)
accuracy5=evaluate_acc(test_y,y5)

print('The MLP model(ReLU activation function, 2 hidden layers,128 hidden units, 0.09 learning rate, 1000 maximum iterations) has accuracy:')
# print(f'train size=0: {accuracy1}')
print(f'train size=10: {accuracy2}')
print(f'train size=100: {accuracy3}')
print(f'train size=1000: {accuracy4}')
print(f'train size=10000: {accuracy5}')